In [7]:
import numpy as np 

classes=np.array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,])

shipIndex=np.where(classes==0)[0]


In [9]:
print(shipIndex)
print(shipIndex[1])

[ 1 11]
11


In [12]:
for i in shipIndex:
    print(i)
    

1
11
